# Direct Access to DAACs
Authors: Alex Mandel (Development Seed), Brian Freitag (NASA MSFC), Jamison French (Development Seed)

Description: In this tutorial, we demonstrate how to assume the MAAP data reader role to access specific DAAC buckets.

***This tutorial demonstrates a temporary workaround to the MAAP's temporary S3 credentials for these DAAC buckets***.

## Run This Notebook
To access and run this tutorial within MAAP's Algorithm Development Environment (ADE), please refer to the "Getting started with the MAAP" section of our documentation.

Disclaimer: it is highly recommended to run a tutorial within MAAP's ADE, which already includes packages and permissions specific to MAAP, such as maap-py. Running the tutorial outside of the MAAP ADE may lead to errors.

## Additional Resources
- [Searching Granules in CMR](../search/granules.ipynb)
- [Searching Collections in CMR](../search/granules.ipynb)
- [Package: fsspec s3fs](https://s3fs.readthedocs.io/en/latest/)

## Importing Packages
If the packages below are not installed already, uncomment the following cell

In [1]:
%pip install h5py fsspec s3fs rioxarray --quiet

Note: you may need to restart the kernel to use updated packages.


In [2]:
import h5py
import boto3
import botocore
import fsspec
import rioxarray
import matplotlib.pyplot as plt

from maap.maap import MAAP

maap = MAAP(maap_host="api.maap-project.org")

## Access The Data
We'll create a couple helper functions to setup the assumed role session and view the data.

In [3]:
def s3_access(ssm_parameter_name):
    # Create a session using your current credentials
    session = boto3.Session()

    # Retrieve the SSM parameter
    ssm = session.client('ssm', "us-west-2")
    parameter = ssm.get_parameter(
        Name=ssm_parameter_name, 
        WithDecryption=True
    )
    parameter_value = parameter['Parameter']['Value']

    # Assume the new role
    sts = session.client('sts')
    assumed_role_object = sts.assume_role(
        RoleArn=parameter_value,
        RoleSessionName='TutorialSession'
    )

    # From the response that contains the assumed role, get the temporary 
    # credentials that can be used to make subsequent API calls
    credentials = assumed_role_object['Credentials']

    return fsspec.filesystem(
        "s3",
        key=credentials['AccessKeyId'],
        secret=credentials['SecretAccessKey'],
        token=credentials['SessionToken']
    )

def tif_info(s3_obj):
    with s3.open(s3_obj) as obj:
        da = rioxarray.open_rasterio(obj)
        return da

Initialize the assumed role session

In [4]:
s3 = s3_access("/iam/maap-data-reader")

### NSIDC DAAC Access

In [5]:
nsidc_object = "s3://nsidc-cumulus-prod-protected/ATLAS/ATL08/006/2023/06/21/ATL08_20230621235543_00272011_006_01.h5"
with s3.open(nsidc_object) as f:
    ATL08_data = h5py.File(f, "r")
    print(ATL08_data.keys())

<KeysViewHDF5 ['METADATA', 'ancillary_data', 'ds_geosegments', 'ds_metrics', 'ds_surf_type', 'gt1l', 'gt1r', 'gt2l', 'gt2r', 'gt3l', 'gt3r', 'orbit_info', 'quality_assessment']>


### ORNL DAAC Access

In [6]:
ornl_object = "s3://ornl-cumulus-prod-protected/gedi/GEDI_L4B_Gridded_Biomass/data/GEDI04_B_MW019MW138_02_002_05_R01000M_PS.tif"
tif_info(ornl_object)

<xarray.DataArray (band: 1, y: 14616, x: 34704)>
[507233664 values with dtype=uint8]
Coordinates:
  * band         (band) int64 1
  * x            (x) float64 -1.737e+07 -1.737e+07 ... 1.737e+07 1.737e+07
  * y            (y) float64 7.314e+06 7.313e+06 ... -7.313e+06 -7.314e+06
    spatial_ref  int64 0
Attributes:
    AREA_OR_POINT:       Area
    OVR_RESAMPLING_ALG:  MODE
    _FillValue:          0
    scale_factor:        1.0
    add_offset:          0.0
    long_name:           Prediction Stratum

### GES DISC Access

In [7]:
ges_disc_object = "s3://gesdisc-cumulus-prod-protected/Landslide/Global_Landslide_Nowcast.1.1/2020/Global_Landslide_Nowcast_v1.1_20201231.tif"
tif_info(ges_disc_object)

<xarray.DataArray (band: 1, y: 14400, x: 43200)>
[622080000 values with dtype=uint8]
Coordinates:
  * band         (band) int64 1
  * x            (x) float64 -180.0 -180.0 -180.0 -180.0 ... 180.0 180.0 180.0
  * y            (y) float64 60.0 59.99 59.98 59.97 ... -59.98 -59.99 -60.0
    spatial_ref  int64 0
Attributes:
    AREA_OR_POINT:       Area
    STATISTICS_MAXIMUM:  2
    STATISTICS_MEAN:     nan
    STATISTICS_MINIMUM:  0
    STATISTICS_STDDEV:   nan
    _FillValue:          255
    scale_factor:        1.0
    add_offset:          0.0

### LP DAAC Access

In [8]:
lp_object = "s3://lp-prod-protected/HLSL30.020/HLS.L30.T56JMN.2023225T234225.v2.0/HLS.L30.T56JMN.2023225T234225.v2.0.B11.tif"
tif_info(lp_object)

<xarray.DataArray (band: 1, y: 3660, x: 3660)>
[13395600 values with dtype=int16]
Coordinates:
  * band         (band) int64 1
  * x            (x) float64 4e+05 4e+05 4e+05 ... 5.097e+05 5.097e+05 5.097e+05
  * y            (y) float64 -3.2e+06 -3.2e+06 -3.2e+06 ... -3.31e+06 -3.31e+06
    spatial_ref  int64 0
Attributes: (12/37)
    ACCODE:                    Lasrc; Lasrc
    add_offset:                0.0
    AREA_OR_POINT:             Area
    arop_ave_xshift(meters):   0, 0
    arop_ave_yshift(meters):   0, 0
    arop_ncp:                  0, 0
    ...                        ...
    TIRS_SSM_POSITION_STATUS:  ESTIMATED; ESTIMATED
    ULX:                       399960
    ULY:                       -3199980
    unit:                      Celsius
    USGS_SOFTWARE:             LPGS_16.3.0
    _FillValue:                -9999